# HarvestStat Data Profiling - Niger

In [1]:
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:32631' # Niger
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/NE_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#read in username and password
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Niger',
    'product': ['R011','R012','R013','R014','R015','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
# ----------------------------------------------- #

###################################################
# The current submitted data has an issue on start_date. Here we manually correct it.
idx = df['start_date'].apply(lambda x: x[-5:]) == '10-01'
df.loc[idx, 'start_date'] = df.loc[idx, 'start_date'].apply(lambda x: x[:-5] + '12-01')
###################################################

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1972	 | NE1972A1 | 7	| NE1972A2	| 35	| nan	| 0	|
| 1992	 | NE1992A1 | 8	| NE1992A2	| 36	| nan	| 0	|
| 2001	 | NE2001A1 | 8	| NE2001A2	| 37	| nan	| 0	|
| 2012	 | NE2012A1 | 8	| NE2012A2	| 74	| nan	| 0	|
| 2018	 | NE2018A1 | 8	| NE2018A2	| 67	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Shape file correction
# Correct the 2012 admin file for Nigeria to include Agadez Commune
NE_Admin2_2012.loc[NE_Admin2_2012.FNID=='NE2012A20109','ADMIN2'] = 'Agadez Commune'
NE_Admin2_2012.loc[NE_Admin2_2012.FNID=='NE2012A20109','ADMIN2FR'] = 'Agadez Commune'
NE_Admin2_2012.loc[NE_Admin2_2012.FNID=='NE2012A20109','ALIASES'] = 'None'

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual correction
# a) Season name
df.loc[df['season_name'] == 'Main season', 'season_name'] = 'Main'
# b) order of magnitude error in Agadaz commune right after it was creted
df.loc[(df.fnid=='NE2012A20109')&(pd.to_datetime(df.season_date).dt.year==2013)&(df.indicator=='Area Planted'),'value'] = \
df.loc[(df.fnid=='NE2012A20109')&(pd.to_datetime(df.season_date).dt.year==2013)&(df.indicator=='Area Planted'),'value']/10
df.loc[(df.fnid=='NE2012A20109')&(pd.to_datetime(df.season_date).dt.year==2013)&(df.indicator=='Quantity Produced'),'value'] = \
df.loc[(df.fnid=='NE2012A20109')&(pd.to_datetime(df.season_date).dt.year==2013)&(df.indicator=='Quantity Produced'),'value']/10

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([NE_Admin2_1972,NE_Admin2_1992,NE_Admin2_2001,NE_Admin2_2012,NE_Admin2_2018], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 26,980
Removed 2,394 "Missing Value" points
8,260/8,348 "Area Planted" points are retained.
8,163/8,348 "Quantity Produced" points are retained.
8,163/8,348 "Yield" points are retained.
0/1,936 "Area Harvested" points are retained.
Current data points: 24,586

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1980 - 2021
15 grain types are found: Bambara groundnut, Cassava, Cowpeas (Mixed), Fonio, Groundnuts (In Shell), Maize (Corn), Millet, Okras (Fresh), Onions, Rice (Paddy), Sesame Seed, Sorghum, Sorrel, Tigernut, Tomato
1 seasons are found: Main (06-01)
1 crop production system are found: none
Data sources include:
[1] Ministere de l'Agriculture, Niger --- FEWS NET Agro Maps, Niger
[2] Ministere de l'Agriculture, Niger --- Official agricultural statistics, Niger
[3] Ministere de l'Agriculture, Niger --- Resultats definitifs campagne agricole, Niger

## Niger
Niger crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-niger.png?itok=UBgvvRXP)

![USDA](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/wafrica_ng_calendar.png)

- comparison of boundaries

<img src="../figures/NE_admin_shapes.png"  width="60%">

- In Niger, basd on the FEWS NET data, there were four times of changes in administrative units: 1972, 1992, 2012, and 2018.

| year	 | Admin1   | # units   | Admin2   | # units   |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1972	 | NE1972A1 | 7	| NE1972A2	| 35	|
| 1992	 | NE1992A1 | 8	| NE1992A2	| 36	|
| 2001	 | NE2001A1 | 8	| NE2001A2	| 37	|
| 2012	 | NE2012A1 | 8	| NE2012A2	| 74	|
| 2018	 | NE2018A1 | 8	| NE2018A2	| 67	|

- The data is all reported in a single main season
- While there is a shapefile for 2018, there is no crop data associated with the 2018 shapefile, so we use 2012 as the most recent shapefile

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = NE_Admin2_2012.copy().to_crs('epsg:32631')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

NE2001A20704:	"Matameye" (FDW) is changed to "Kantche" (shapefile).
NE2012A20311:	"Dioundiou" (FDW) is changed to "Dioudiou" (shapefile).


In [5]:
# Link admin boundaries ------------------------- #
link_2001, over_2001 = FDW_PD_CreateAdminLink(NE_Admin2_2001, NE_Admin2_2012, 'ADMIN2', 'ADMIN2', prod, epsg)
#assert all(np.unique([v['method'] for k,v in link_2001.items()]) == 'PBR')
link_1992, over_1992 = FDW_PD_CreateAdminLink(NE_Admin2_1992, NE_Admin2_2012, 'ADMIN2', 'ADMIN2', prod, epsg)
#assert all(np.unique([v['method'] for k,v in link_1992.items()]) == 'PBR')
link_1972, over_1972 = FDW_PD_CreateAdminLink(NE_Admin2_1972, NE_Admin2_2012, 'ADMIN2', 'ADMIN2', prod, epsg)
#assert all(np.unique([v['method'] for k,v in link_1972.items()]) == 'PBR')


# Crop specific ratios
link_ratio_1972 = FDW_PD_RatioAdminLink(link_1972, prod, over_1972, mdx_pss)
link_ratio_1992 = FDW_PD_RatioAdminLink(link_1992, prod, over_1992, mdx_pss)
link_ratio_2001 = FDW_PD_RatioAdminLink(link_2001, prod, over_2001, mdx_pss)

# Merge link_ratio
assert link_ratio_1992.keys() == link_ratio_2001.keys()
assert link_ratio_1972.keys() == link_ratio_2001.keys()

link_merged = [link_ratio_1972, link_ratio_1992, link_ratio_2001]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=True)
# ----------------------------------------------- #

CBR is considered for 'NE2001A20103' as no record found in: ['NE2012A20103']
CBR is considered for 'NE2001A20203' as no record found in: ['NE2012A20209']
CBR is considered for 'NE2001A20702' as no record found in: ['NE2012A20707']
CBR is considered for 'NE1992A20103' as no record found in: ['NE2012A20103']
CBR is considered for 'NE1992A20203' as no record found in: ['NE2012A20209']
CBR is considered for 'NE1992A20702' as no record found in: ['NE2012A20707']
CBR is considered for 'NE1972A20702' as no record found in: ['NE2012A20707']
CBR is considered for 'NE1972A20203' as no record found in: ['NE2012A20209']
CBR is considered for 'NE1972A20103' as no record found in: ['NE2012A20103']


In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
#I need to be able to look at the source docs here. I think maize planted area was used
#area_new.loc[2019,pd.IndexSlice['NE2012A20804',:,'Millet','Main season',:,:,'none']] = 2000
#area_new.loc[2019,pd.IndexSlice['NE2012A20805',:,'Millet','Main season',:,:,'none']] = 4000

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {
    'Bambara groundnut': 'Bambara groundnut',
    'Cassava': 'Cassava',
    'Cowpeas (Mixed)': 'Cowpeas',
    'Fonio': 'Fonio',
    'Groundnuts (In Shell)': 'Groundnuts (In Shell)',
    'Maize (Corn)': 'Maize',
    'Millet': 'Millet',
    'Okras (Fresh)': 'Okras',
    'Onions': 'Onions',
    'Rice (Paddy)': 'Rice',
    'Sesame Seed': 'Sesame Seed',
    'Sorghum': 'Sorghum',
    'Sorrel': 'Sorrel',
    'Tigernut': 'Tigernut',
    'Tomato': 'Tomato'
}
stack['product'] = stack['product'].replace(product_category_custom)
for f, r in link_ratio.items():
    mdx = pd.MultiIndex.from_frame(r.index.to_frame().reset_index(drop=True).replace(product_category_custom))
    r.index = mdx
    link_ratio[f] = r
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Niger']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_NE.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_npz('../data/crop/adm_crop_production_NE_ratio.npz', link_ratio)

All [season_name, product, crop_production_system] are in the external crop calendar.
../data/crop/adm_crop_production_NE.csv is saved.
../data/crop/adm_crop_production_NE_ratio.npz is saved.


## Visualization of production data

In [7]:
# Bar chart of national crop production
country_iso, country_name = 'NE', 'Niger'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize', 'Fonio','Millet', 'Rice','Sorghum']
for season_name in ['Main']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

../figures/NE_bar_natgrainprod_Main.png is saved.


In [8]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'NE', 'Niger'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Millet','Main'],
    ['Sorghum','Main'],
    ['Maize','Main'],
    ['Rice','Main'],
    ['Fonio','Main'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

../figures/NE_line_pay_Millet_Main.png is saved.


../figures/NE_line_pay_Sorghum_Main.png is saved.


../figures/NE_line_pay_Maize_Main.png is saved.


../figures/NE_line_pay_Rice_Main.png is saved.


../figures/NE_line_pay_Fonio_Main.png is saved.
